In [3]:
import re
import requests
from bs4 import BeautifulSoup, Comment
import pandas as pd
import numpy as np
from concurrent.futures import ThreadPoolExecutor

# For Plotting
import plotly.graph_objects as go
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

import warnings
warnings.filterwarnings("ignore")

Data Extraction¶
I have tried to show how we can scrap the data in some basic steps. I Have used the Air Matters for scraping the data of some cities mentioned in this site.
https://air-quality.com/

In [4]:
# Example with one url
city_url="https://air-quality.com/place/india/akasahebpeta/e683280a?lang=en&standard=naqi_in"
city_url

'https://air-quality.com/place/india/akasahebpeta/e683280a?lang=en&standard=naqi_in'

In [5]:
city_response = requests.get(city_url)
city_soup = BeautifulSoup(city_response.content, "html.parser")

# Extract the city name and state name
city_name = city_soup.find("h2").text.strip()
state_name = city_soup.find("p").text.strip().split(", ")[1]
print(f"{state_name}, {city_name}")

Andhra Pradesh, Akasahebpeta


In [6]:
# Extract AQI value from HTML comment
aqi_comment = city_soup.find(text=lambda text: isinstance(text, Comment) and "indexValue" in text)
aqi_value = re.search(r"\d+", aqi_comment).group() if aqi_comment else np.NaN
print(aqi_value)

25


In [7]:
# Extract pollutant information
pollutants = city_soup.find_all("div", class_="pollutant-item")

pollutant_data = {}
for pollutant in pollutants:
    name = pollutant.find("div", class_="name").text.strip()
    value = pollutant.find("div", class_="value").text.strip()
    pollutant_data[name] = value

print(pollutant_data.get("PM2.5"), pollutant_data.get("PM10"), pollutant_data.get("O3"), pollutant_data.get("SO2"), pollutant_data.get("CO"))

14.6 22.3 None None None


In [8]:
# Extract weather information
temperature = city_soup.find("div", class_="temperature").text.strip().replace("℃", "")
humidity = city_soup.find("div", class_="humidity").text.strip().replace("%", "")
wind_speed = city_soup.find("div", class_="wind").text.strip().replace("kph", "")
type = city_soup.find("div", class_="level").text.strip()
print(temperature,humidity,wind_speed,type)

33 57 12  Good


In [9]:
# Check if GPU is available and use it if possible
use_gpu = False
try:
    import cupy as cp
    use_gpu = True
except ImportError:
    pass